In [1]:
# Importing necessary packages
import json
import os
import pandas as pd
import numpy as np

In [2]:
# Getting path to our directory/folder
directory = os.getcwd()

In [3]:
# Loading the JSON file which was downloaded from postman.com
with open(directory + "\\response_AAPL.json", "r") as f:
    data = json.load(f)

In [4]:
# Saving the loaded content in a variable
ts_1 = data

In [5]:
# Creating a class ScriptData that fetches stock data
class ScriptData:
    # Contructor for overloading
    def __init__(self, script):
        self.script = script
        self.fetch_intraday_data(self.script)
        self.df = self.convert_intraday_data(self.data)
        self.df["symbol"] = self.script

    # Fetching raw stock data
    def fetch_intraday_data(self, script):
        self.data = ts_1
        return self.data
    
    # This method is used to fetch Intraday Historical day data
    # and convert it to a Data Frame
    def fetch_historical_data(self):
        df_csv = pd.read_csv(directory + "\\extended_intraday_GOOGL.csv")
        df_csv = df_csv.rename(columns={'time': 'timestamp'})
        return df_csv
    
    # Tis method converts the raw data that has been fetched
    # by fetch_intraday_data method and converts it into a Data Frame
    def convert_intraday_data(self, data):
        self.data_list = []
        for timestamp, items in data["Time Series (5min)"].items():
            open_item = float(items["1. open"])
            high_item = float(items["2. high"])
            low_item = float(items["3. low"])
            close_item = float(items["4. close"])
            volume = int(items["5. volume"])
            self.data_list.append([timestamp, open_item, high_item, low_item, close_item, volume])
        df = pd.DataFrame(self.data_list, columns=[ "timestamp", "open", "high", "low", "close", "volume"])
        df["timestamp"] = pd.to_datetime(df["timestamp"])
        return df

    # This method calculates moving averages in close/close_data
    # and saves it in indicator column
    def indicator1(self, timestamp, dataf, timeperiod= 5):
        indicator_data = []
        final_output = {"timestamp": [], "indicator" : []}
        upcount = 0
        downcount = timeperiod - 1
        # Below part is where the moving average calculation takes place
        while downcount < len(dataf):
            cal_temp = 0
            # This for_loop loops from range upcount to downcount + 1
            # In this case it goes from 0 to 5+1 that means 0 - 5 and 
            # both upcount and downcount increases for each while loop
            for i in range(upcount, downcount + 1):
                cal_temp = float(dataf["close"][i]) + cal_temp
            final_temp = cal_temp/timeperiod
            # Appending the calculated data to a list
            indicator_data.append(final_temp)
            if downcount + 1 >= len(dataf):
                break
            upcount += 1
            downcount += 1
        # Appending timestamp column to a dictionary
        for i in range(0, len(dataf)):
            final_output["timestamp"].append(dataf["timestamp"][i])
        # Appending moving average(final_temp) column to a dictionary
        for i in range(0, len(dataf)):
            if i < timeperiod - 1:
                final_output["indicator"].append("NaN")
            else:
                final_output["indicator"].append(
                    indicator_data[i-timeperiod + 1])
        # Finally converting the dictionary to a DataFrame with columns
        # "timestamp" and "indicator"
        df_1 = pd.DataFrame(final_output)
        df_1.fillna(value= np.nan, inplace = True)
        return df_1
    
    # Overloading methods
    def __getitem__(self, index):
        return self.df.loc[index]

    def __setitem__(self, index, value):
        self.df.iloc[index] = value

    def __contains__(self, value):
        return value in self.df['symbol'].values

In [6]:
# Creating an object for calls ScriptData and passing parameter
# as it needs to be initiated for __init(self, script)
script_data = ScriptData(ts_1)

In [7]:
# Calling convert_intraday_data() method with
# raw data present in fetch_intraday_data()
script_data.convert_intraday_data(script_data.fetch_intraday_data(ts_1))

,timestamp,open,high,low,close,volume
0,2023-02-14 20:00:00,152.79,152.850,152.76,152.84,9429
1,2023-02-14 19:55:00,152.75,152.770,152.73,152.77,3596
2,2023-02-14 19:50:00,152.75,152.760,152.70,152.71,4770
3,2023-02-14 19:45:00,152.84,152.870,152.75,152.76,7379
4,2023-02-14 19:40:00,152.87,152.870,152.84,152.84,1795
...,...,...,...,...,...,...
95,2023-02-14 12:00:00,151.56,151.830,151.36,151.70,518882
96,2023-02-14 11:55:00,151.53,151.600,151.17,151.55,496456
97,2023-02-14 11:50:00,151.13,151.720,151.10,151.54,494145
98,2023-02-14 11:45:00,151.35,151.386,151.04,151.14,490943


In [8]:
# Printing out the contents of indicator1 with moving average
script_data.indicator1(timestamp= script_data.df["timestamp"], dataf = script_data.df)

,timestamp,indicator
0,2023-02-14 20:00:00,NaN
1,2023-02-14 19:55:00,NaN
2,2023-02-14 19:50:00,NaN
3,2023-02-14 19:45:00,NaN
4,2023-02-14 19:40:00,152.784
...,...,...
95,2023-02-14 12:00:00,151.84
96,2023-02-14 11:55:00,151.746
97,2023-02-14 11:50:00,151.66
98,2023-02-14 11:45:00,151.522


In [9]:
# Printing the historical data which is stored in excel file
script_data.fetch_historical_data()

,timestamp,open,high,low,close,volume
0,2023-02-14 20:00:00,94.18,94.1800,94.11,94.12,9174
1,2023-02-14 19:55:00,94.23,94.2300,94.18,94.18,2882
2,2023-02-14 19:50:00,94.24,94.2400,93.50,93.50,15924
3,2023-02-14 19:45:00,94.26,94.2899,94.20,94.20,5486
4,2023-02-14 19:40:00,94.26,94.3200,94.25,94.32,5043
...,...,...,...,...,...,...
3853,2023-01-17 04:25:00,91.82,91.8200,91.60,91.72,3784
3854,2023-01-17 04:20:00,91.86,91.8800,91.86,91.88,1980
3855,2023-01-17 04:15:00,91.90,91.9700,91.87,91.87,3610
3856,2023-01-17 04:10:00,91.82,91.9300,91.80,91.90,2016


In [10]:
# Creating a class Strategy which processes on Historical day data
class Strategy:
    # Constructor
    def __init__(self, script_data):
        self.script_data = script_data
        
    # Fetching historical data data frame from ScriptData class
    # Changing the column names of 
    # "indicator" to "indicator_data"
    # "close" to "close_data", here we directly store data from close
    def fetch_intraday_historical_day_data(self):
        a = self.script_data.fetch_historical_data()
        close_values = [i for i in a["close"]]
        b = self.script_data.indicator1(a["timestamp"], a)
        b = b.rename(columns={'indicator': 'indicator_data'})
        for i in range(0, len(a["close"])):
            b["close_data"] = close_values
        return b

    # In this method we calculate BUY, SELL and NO SIGNAL based on 
    # "indicator_data" and "close_data" and append it to a column called "signal"
    def get_signal(self):
        a = self.script_data.fetch_historical_data()
        close_values = [i for i in a["close"]]
        b = self.script_data.indicator1(a["timestamp"], a)
        b = b.rename(columns={'indicator': 'indicator_data'})
        for i in range(0, len(a["close"])):
            b["close_data"] = close_values
        c = {"timestamp": [], "signal" : []}
        for i in range(0, len(b)):
            if float(b["indicator_data"].iloc[i]) < float(b["close_data"].iloc[i]):
                c["timestamp"].append(b["timestamp"][i])
                c["signal"].append("SELL")
            elif float(b["indicator_data"].iloc[i]) > float(b["close_data"].iloc[i]):
                c["timestamp"].append(b["timestamp"][i])
                c["signal"].append("BUY")
            else:
                c["timestamp"].append(b["timestamp"][i])
                c["signal"].append("NO SIGNAL")
        df_03 = pd.DataFrame(c)
        # Removing any row with "NO SIGNAL" and resetting the index
        df_03 = df_03[df_03['signal'] != 'NO SIGNAL'].reset_index(drop=True)
        # Uncomment this incase you want to get the output in excel file
        # df_03.to_excel('output.xlsx', index=True)
        return df_03

In [11]:
# Creating an object for Strategy and passing an argument/parameter
strategy_data = Strategy(script_data)

In [12]:
# Getting the modified historical to calculate BUY, SELL and GET SIGNAL
strategy_data.fetch_intraday_historical_day_data()

,timestamp,indicator_data,close_data
0,2023-02-14 20:00:00,NaN,94.12
1,2023-02-14 19:55:00,NaN,94.18
2,2023-02-14 19:50:00,NaN,93.50
3,2023-02-14 19:45:00,NaN,94.20
4,2023-02-14 19:40:00,94.064,94.32
...,...,...,...
3853,2023-01-17 04:25:00,91.7,91.72
3854,2023-01-17 04:20:00,91.73,91.88
3855,2023-01-17 04:15:00,91.772,91.87
3856,2023-01-17 04:10:00,91.82,91.90


In [13]:
# Printing get_signal()
strategy_data.get_signal()

,timestamp,signal
0,2023-02-14 19:40:00,SELL
1,2023-02-14 19:35:00,SELL
2,2023-02-14 19:30:00,SELL
3,2023-02-14 19:20:00,BUY
4,2023-02-14 19:15:00,BUY
...,...,...
3832,2023-01-17 04:25:00,SELL
3833,2023-01-17 04:20:00,SELL
3834,2023-01-17 04:15:00,SELL
3835,2023-01-17 04:10:00,SELL


In [14]:
# Checking if AAPL is in the data we are processing
"AAPL" in data["Meta Data"]["2. Symbol"]

True